In [51]:
import os
import sys
import pandas as pd
from pathlib import Path

import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from fairlearn.adversarial import AdversarialFairnessClassifier

sys.path.append(os.path.abspath('..'))
from util import train_model_on_datasets

In [5]:
# load datasets
data_path = Path(os.getcwd()).parent.parent / "data" / "dataset_diabetes" / "clsf_data"

In [6]:
target_variable = "readmit_30_days"
sensitive_attribute = "race"

In [7]:
X_test = pd.read_csv(data_path / "X_test_split.csv")
X_A_test = pd.read_csv(data_path / "X_A_test_split.csv")
Y_test = pd.read_csv(data_path / "Y_test_split.csv")[target_variable]
A_test = pd.read_csv(data_path / "A_test_split.csv")[sensitive_attribute]

X_train = pd.read_csv(data_path / "X_train_split.csv")
X_A_train = pd.read_csv(data_path / "X_A_train_split.csv")
Y_train = pd.read_csv(data_path / "Y_train_split.csv")[target_variable]
A_train = pd.read_csv(data_path / "A_train_split.csv")[sensitive_attribute]

X_train_res_target_wos = pd.read_csv(data_path / "X_train_res_target_wos.csv")
Y_train_res_target_wos = pd.read_csv(data_path / "Y_train_res_target_wos.csv")[target_variable]
A_train_res_target_wos = pd.read_csv(data_path / "A_train_res_target_wos.csv").iloc[:, 0]

X_A_train_res_target_ws = pd.read_csv(data_path / "X_A_train_res_target_ws.csv")
Y_train_res_target_ws = pd.read_csv(data_path / "Y_train_res_target_ws.csv")[target_variable]
A_train_res_target_ws = pd.read_csv(data_path / "A_train_res_target_ws.csv").iloc[:, 0]

X_train_res_sensitive_wos = pd.read_csv(data_path / "X_train_res_sensitive_wos.csv")
Y_train_res_sensitive_wos = pd.read_csv(data_path / "Y_train_res_sensitive_wos.csv")[target_variable]
A_train_res_sensitive_wos = pd.read_csv(data_path / "A_train_res_sensitive_wos.csv").iloc[:, 0]

X_A_train_res_sensitive_ws = pd.read_csv(data_path / "X_A_train_res_sensitive_ws.csv")
Y_train_res_sensitive_ws = pd.read_csv(data_path / "Y_train_res_sensitive_ws.csv")[target_variable]
A_train_res_sensitive_ws = pd.read_csv(data_path / "A_train_res_sensitive_ws.csv").iloc[:, 0]

X_train_res_multiv_wos = pd.read_csv(data_path / "X_train_res_multiv_wos.csv")
Y_train_res_multiv_wos = pd.read_csv(data_path / "Y_train_res_multiv_wos.csv")[target_variable]
A_train_res_multiv_wos = pd.read_csv(data_path / "A_train_res_multiv_wos.csv").iloc[:, 0]

X_A_train_res_multiv_ws = pd.read_csv(data_path / "X_A_train_res_multiv_ws.csv")
Y_train_res_multiv_ws = pd.read_csv(data_path / "Y_train_res_multiv_ws.csv")[target_variable]
A_train_res_multiv_ws = pd.read_csv(data_path / "A_train_res_multiv_ws.csv").iloc[:, 0]

In [24]:
A_train.to_frame().values

array([['Caucasian'],
       ['Caucasian'],
       ['Other'],
       ...,
       ['Caucasian'],
       ['Caucasian'],
       ['AfricanAmerican']], dtype=object)

In [27]:
class PandasDataSet(TensorDataset):

    def __init__(self, *dataframes):
        tensors = (self._df_to_tensor(df) for df in dataframes)
        super(PandasDataSet, self).__init__(*tensors)

    def _df_to_tensor(self, df):
        if isinstance(df, pd.Series):
            df = df.to_frame()
        return torch.tensor(df.values).float()

train_data = PandasDataSet(X_train, Y_train, pd.get_dummies(A_train))
test_data = PandasDataSet(X_test, Y_test, pd.get_dummies(A_test))

In [34]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, drop_last=True)

print('# training samples:', len(train_data))
print('# batches:', len(train_loader))

# training samples: 81410
# batches: 2544


In [28]:
class Classifier(nn.Module):

    def __init__(self, n_features, n_hidden=32, p_dropout=0.2):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(n_features, n_hidden),
            nn.ReLU(),
            nn.Dropout(p_dropout),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Dropout(p_dropout),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Dropout(p_dropout),
            nn.Linear(n_hidden, 1),
        )

    def forward(self, x):
        return F.sigmoid(self.network(x))

In [32]:
X_train.shape[1]

127

In [33]:
clf = Classifier(n_features=X_train.shape[1])
clf_criterion = nn.BCELoss()
clf_optimizer = optim.Adam(clf.parameters())

In [35]:
N_CLF_EPOCHS = 2

for epoch in range(N_CLF_EPOCHS):
    for x, y, _ in train_loader:
        clf.zero_grad()
        p_y = clf(x)
        loss = clf_criterion(p_y, y)
        loss.backward()
        clf_optimizer.step()

c:\Users\Tanya\Desktop\TANYA\FMI\Masters\thesis\src\env\lib\site-packages\torch\autograd\__init__.py:251: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ..\c10\cuda\CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [45]:
class Adversary(nn.Module):

    def __init__(self, n_sensitive, n_hidden=32):
        super(Adversary, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(1, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_sensitive),
        )

    def forward(self, x):
        return F.sigmoid(self.network(x))

lambdas = torch.Tensor([200, 30, 30, 30])
adv = Adversary(n_sensitive=pd.get_dummies(A_train).shape[1])
adv_criterion = nn.BCELoss(reduce=False)
adv_optimizer = optim.Adam(adv.parameters())

N_ADV_EPOCHS = 5

for epoch in range(N_ADV_EPOCHS):
    for x, _, z in train_loader:
        adv.zero_grad()
        p_y = clf(x).detach()
        p_z = adv(p_y)
        loss = (adv_criterion(p_z, z) * lambdas).mean()
        loss.backward()
        adv_optimizer.step()

c:\Users\Tanya\Desktop\TANYA\FMI\Masters\thesis\src\env\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [52]:
N_EPOCH_COMBINED = 165

for epoch in range(1, N_EPOCH_COMBINED):
    print(epoch)

    # Train adversary
    for x, y, z in train_loader:
        adv.zero_grad()
        p_y = clf(x)
        p_z = adv(p_y)
        loss_adv = (adv_criterion(p_z, z) * lambdas).mean()
        loss_adv.backward()
        adv_optimizer.step()

    # Train classifier on single batch
    for x, y, z in train_loader:
        pass  # Ugly way to get a single batch
    clf.zero_grad()
    p_y = clf(x)
    p_z = adv(p_y)
    loss_adv = (adv_criterion(p_z, z) * lambdas).mean()
    clf_loss = clf_criterion(p_y, y) - (adv_criterion(adv(p_y), z) * lambdas).mean()
    clf_loss.backward()
    clf_optimizer.step()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164


===========

In [53]:
mitigator = AdversarialFairnessClassifier(
    backend="torch",
    predictor_model=[50, "leaky_relu"],
    adversary_model=[3, "leaky_relu"],
    batch_size=2 ** 8,
    progress_updates=0.5,
    random_state=123,
)

In [ ]:
mitigator.fit(X_prep_train, Y_train, sensitive_features=Z_train)